# CS 109B/Data Science: Final Project - Milestone 3
**Harvard University**<br>
**Spring 2017**<br>
**Date: ** April 19, 2017 

In [118]:
%matplotlib inline
import os
import sys
import nltk
import json
import itertools
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp
import random
import time
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from collections import Counter
from unidecode import unidecode
from nltk.corpus import stopwords
from collections import OrderedDict
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold as sk_kfold
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV as LogRegCV
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.model_selection import GridSearchCV

plt.style.use('ggplot')

# Part I: Defining Dataset

### *Step 1: Clean and explore the data*

In [119]:
# Load data
with open('themoviedb-5k-sample-4-15-2017') as json_data:
    d = json.load(json_data)
raw = pd.DataFrame(d)

# Sample size and data visualization
print(raw.shape)
raw.head()

(5000, 29)


,adult,backdrop_path,belongs_to_collection,budget,cast,crew,genre_ids,genres,homepage,id,...,revenue,reviews,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,None,None,0,[],"[{u'name': u'Bobby Sheehan', u'department': u'...",[99],"[{u'name': u'Documentary', u'id': 99}]",https://www.boughtmovie.net,328380,...,0,[],93.0,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Bought,False,0.0,0
1,False,None,None,0,[],"[{u'name': u'Nancy Donnelly', u'department': u...",[99],"[{u'id': 99, u'name': u'Documentary'}]",,112052,...,0,[],60.0,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Egypt Underworld,False,3.8,2
2,False,/jWgtb7CRy3jWoXiz0IK352MG8b5.jpg,None,0,"[{u'name': u'Zhang Jin', u'character': u'Chau ...","[{u'name': u'Douglas Kung Cheung-Tak', u'depar...",[28],"[{u'id': 28, u'name': u'Action'}]",http://www.hkcinemagic.com/en/movie.asp?id=6627,201706,...,0,[],1.0,"[{u'name': u'广州话 / 廣州話', u'iso_639_1': u'cn'},...",Released,,Chinese Heroes,False,0.0,0
3,False,None,None,0,"[{u'name': u'Erika Eleniak', u'character': u'J...","[{u'name': u'Brian Katkin', u'department': u'D...","[28, 18, 53]","[{u'id': 28, u'name': u'Action'}, {u'id': 18, ...",,61803,...,0,[],92.0,"[{u'name': u'English', u'iso_639_1': u'en'}]",Released,he quake of the century... get ready to rumble...,Shakedown,False,3.0,2
4,False,/sHH63HMAQsKLl0vpJ6AxulmkL92.jpg,None,0,"[{u'name': u'Kay Panabaker', u'character': u'J...","[{u'name': u'Kevin Tancharoen', u'department':...","[10402, 35, 18, 10749]","[{u'id': 10402, u'name': u'Music'}, {u'id': 35...",,28665,...,0,[],107.0,"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,Fame,False,5.4,109


In [120]:
# Keeping only genres, overviews, and reviews
raw = raw[['genres', 'reviews', 'overview', 'id']].copy()
print(raw.shape)
raw.head()

(5000, 4)


,genres,reviews,overview,id
0,"[{u'name': u'Documentary', u'id': 99}]",[],Modern industrialization is no longer about st...,328380
1,"[{u'id': 99, u'name': u'Documentary'}]",[],The documentary is an analysis of the fascinat...,112052
2,"[{u'id': 28, u'name': u'Action'}]",[],A good-natured kung fu kid gets caught up in c...,201706
3,"[{u'id': 28, u'name': u'Action'}, {u'id': 18, ...",[],In Los Angeles a deadly plague called the 'Pan...,61803
4,"[{u'id': 10402, u'name': u'Music'}, {u'id': 35...",[],"An updated version of the 1980 musical, which ...",28665


#### 1.1. Define Y variable

In [121]:
all_bigrams_ever = []

def bigrams(row):
    return zip(row['genre_lists'], row['genre_lists'][1:])

def col_for_each_bigram(row):
    for bigram in row['genre_bigrams']:            
        all_bigrams_ever.append(tuple(sorted(bigram)))
    return row

def popular_bigram(row):
    for k in row['genre_bigrams']:
        #if bigram_frequency[k] <= 50 and len(row['genre_lists']) != 1:
            #row['popular_bigram'] = 'Other'
        if k in OrderedDict(bigram_frequency.most_common()).keys() and row['popular_bigram'] != k:
            row['popular_bigram'] = k
    if len(row['genre_lists']) == 1:
        row['popular_bigram'] = tuple(row['genre_lists'])
    return row
                
raw['genre_lists'] = raw.apply(lambda x: [y['name'] for y in x['genres']], axis = 1)
for i in range(raw.shape[0]):
    raw.loc[i, 'genre_lists'].sort()

raw['genre_bigrams'] = raw.apply(lambda x: bigrams(x), axis = 1)
raw['popular_bigram'] = np.nan
raw = raw.apply(lambda x: col_for_each_bigram(x), axis = 1)
bigram_frequency = Counter(all_bigrams_ever)

raw = raw.apply(lambda x: popular_bigram(x), axis = 1)
raw.head(n=5)

,genres,reviews,overview,id,genre_lists,genre_bigrams,popular_bigram
0,"[{u'name': u'Documentary', u'id': 99}]",[],Modern industrialization is no longer about st...,328380,[Documentary],[],"(Documentary,)"
1,"[{u'id': 99, u'name': u'Documentary'}]",[],The documentary is an analysis of the fascinat...,112052,[Documentary],[],"(Documentary,)"
2,"[{u'id': 28, u'name': u'Action'}]",[],A good-natured kung fu kid gets caught up in c...,201706,[Action],[],"(Action,)"
3,"[{u'id': 28, u'name': u'Action'}, {u'id': 18, ...",[],In Los Angeles a deadly plague called the 'Pan...,61803,"[Action, Drama, Thriller]","[(Action, Drama), (Drama, Thriller)]","(Drama, Thriller)"
4,"[{u'id': 10402, u'name': u'Music'}, {u'id': 35...",[],"An updated version of the 1980 musical, which ...",28665,"[Comedy, Drama, Music, Romance]","[(Comedy, Drama), (Drama, Music), (Music, Roma...","(Music, Romance)"


genre_big = pd.DataFrame(raw.groupby(['popular_bigram'])['id'].count())
genre_big = genre_big.reset_index()
genre_big.loc[:, 'prop'] = genre_big.loc[:, 'id']/genre_big['id'].sum()
genre_big2 = genre_big[genre_big['prop'] < 0.01]
genre_big.shape, genre_big2.shape

In [123]:
# 
genre_big = pd.DataFrame(raw.groupby(['popular_bigram'])['id'].count())
genre_big = genre_big.reset_index()
genre_big.loc[:, 'prop'] = genre_big.loc[:, 'id']/genre_big['id'].sum()
genre_big2 = genre_big[genre_big['prop'] < 0.01]
genre_big.shape, genre_big2.shape

((176, 3), (153, 3))

In [124]:
raw['popular_bigram2'] = raw['popular_bigram']
for i in range(raw.shape[0]):
    if raw.loc[i, 'popular_bigram'] in list(genre_big2['popular_bigram']):
        raw.loc[i, 'popular_bigram2'] = 'Other'

In [125]:
genre_frequency = raw['popular_bigram2'].value_counts()
genre_frequency

Other                          1538
(Documentary,)                  667
(Drama,)                        462
(Comedy,)                       347
(Drama, Romance)                207
(Horror, Thriller)              183
(Drama, Thriller)               174
(Horror,)                       171
(Comedy, Drama)                 142
(Music,)                        131
(Documentary, Music)             94
(Comedy, Romance)                93
(Science Fiction, Thriller)      91
(Thriller,)                      84
(Mystery, Thriller)              82
(Crime, Drama)                   76
(Drama, Family)                  66
(Animation, Family)              63
(Horror, Science Fiction)        61
(Comedy, Horror)                 57
(Action,)                        56
(Animation,)                     52
(Comedy, Family)                 52
(Crime, Thriller)                51
Name: popular_bigram2, dtype: int64

#### 2.2. Define X variables

Converting overview and reviews to word-frequency vectors

In [130]:
# I realized that sometimes 'overview' was in my top words list; renaming it here 'protects' it
df = raw[['genres', 'reviews', 'overview', 'genre_lists', 'genre_bigrams', 'popular_bigram2']]
df.rename(columns={'overview': 'movie_overview'}, inplace=True)

stops = set(nltk.corpus.stopwords.words('english'))
all_words_ever = []

def add_cols_for_all_words(row):
    if row['movie_overview'] is not None:
        words = nltk.word_tokenize(row['movie_overview'])
        words = [word.lower() for word in words if len(word) > 2 and unidecode(word.lower()) not in stops]
        all_words_ever.extend(words)
    return row

def parse_words(row):
    if row['movie_overview'] is not None:
        words = nltk.word_tokenize(row['movie_overview'])
        words = [word.lower() for word in words if word.lower() in top_words]
        for word in words:
            row[unidecode(word)] += 1        
    return row

In [132]:
df['review_text'] = df.apply(lambda row: [x['content'] for x in row['reviews']], axis = 1)
df.head()

/Users/Fanta/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,genres,reviews,movie_overview,genre_lists,genre_bigrams,popular_bigram2,review_text
0,"[{u'name': u'Documentary', u'id': 99}]",[],Modern industrialization is no longer about st...,[Documentary],[],"(Documentary,)",[]
1,"[{u'id': 99, u'name': u'Documentary'}]",[],The documentary is an analysis of the fascinat...,[Documentary],[],"(Documentary,)",[]
2,"[{u'id': 28, u'name': u'Action'}]",[],A good-natured kung fu kid gets caught up in c...,[Action],[],"(Action,)",[]
3,"[{u'id': 28, u'name': u'Action'}, {u'id': 18, ...",[],In Los Angeles a deadly plague called the 'Pan...,"[Action, Drama, Thriller]","[(Action, Drama), (Drama, Thriller)]","(Drama, Thriller)",[]
4,"[{u'id': 10402, u'name': u'Music'}, {u'id': 35...",[],"An updated version of the 1980 musical, which ...","[Comedy, Drama, Music, Romance]","[(Comedy, Drama), (Drama, Music), (Music, Roma...",Other,[]


In [133]:
df = df.apply(lambda x: add_cols_for_all_words(x), axis = 1)
len(all_words_ever), len(set(all_words_ever))

(153604, 25820)

In [134]:
# Use the top N words
N = 500
top_words = [word[0] for word in Counter(all_words_ever).most_common(N)]
len(top_words)

500

In [135]:
for word in top_words:
    df[unidecode(word)] = 0

df = df.apply(lambda x: parse_words(x), axis = 1)
    
print(df.shape)
df.head()

(5000, 507)


,genres,reviews,movie_overview,genre_lists,genre_bigrams,popular_bigram2,review_text,one,life,film,...,course,break,plays,violence,looks,water,west,trapped,filmmaker,god
0,"[{u'name': u'Documentary', u'id': 99}]",[],Modern industrialization is no longer about st...,[Documentary],[],"(Documentary,)",[],0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[{u'id': 99, u'name': u'Documentary'}]",[],The documentary is an analysis of the fascinat...,[Documentary],[],"(Documentary,)",[],0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[{u'id': 28, u'name': u'Action'}]",[],A good-natured kung fu kid gets caught up in c...,[Action],[],"(Action,)",[],0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,"[{u'id': 28, u'name': u'Action'}, {u'id': 18, ...",[],In Los Angeles a deadly plague called the 'Pan...,"[Action, Drama, Thriller]","[(Action, Drama), (Drama, Thriller)]","(Drama, Thriller)",[],0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[{u'id': 10402, u'name': u'Music'}, {u'id': 35...",[],"An updated version of the 1980 musical, which ...","[Comedy, Drama, Music, Romance]","[(Comedy, Drama), (Drama, Music), (Music, Roma...",Other,[],0,0,0,...,0,0,0,0,0,0,0,0,0,0


### *Step 2: Fitt the model: SVM - kernel: RBF *

In [145]:
# Define Y and X matrixes
X = df[df.columns[-N:]]
y = df['popular_bigram2']
y = y.apply(lambda x: str(x))
X.shape, y.shape

((5000, 500), (5000,))

In [148]:
# Train/test split
n_samples = len(X)
train_indices = np.random.uniform(size=n_samples) > 1. / 3.  #Select two thirds for train

x_train = X[train_indices]
y_train = y[train_indices]

x_test = X[~train_indices]
y_test = y[~train_indices]

Tune Parameters SVM model 

In [156]:
#--------  optimum_cvc
# Function to optimize parameter C on Unweighted SVM
# Input: 
#      x_train (array of train predictors)
#      y_train (array of train responses) 
#      n_folds (float; number of folds for cross validation)    
#      seed (float; fix the random the random state)
#      weight (string; type of weight)
# Output:
#      best_c (optimal value of C)
#      best_k (optimal kernel)
#      results_df (DataFrame for scores, std_scores, per model)   

def optimum_cvc(x_train, y_train, n_folds, seed, weight):
    n_folds = n_folds
    seed = seed
    kfold = sk_kfold(len(y_train), n_folds, random_state=seed)
    scores = list()
    scores_std = list()
    C = list()
    gamma = list()
    
    # To keep track of the best model
    best_score = 0
    for c in np.logspace(-3, 2, 6):
        for g in np.logspace(-3, 2, 6):
            model = SVC(C=c, gamma=g, class_weight = weight)
            model.fit(x_train, y_train)
            i_scores = cross_val_score(model, x_train, y_train, cv=kfold, n_jobs=1)
            scores.append(np.mean(i_scores))
            scores_std.append(np.std(i_scores))
            C.append(c)
            gamma.append(g)
            
            # Update our record of the best parameters see so far
            if np.mean(i_scores) > best_score:
                best_score = np.mean(i_scores)
                best_C = c
                best_gamma = g
    
    # Scores, scores std, for different combinations of C and gamma              
    scores, scores_std = np.array(scores), np.array(scores_std)
    C = np.array(C)
    gamma = np.array(gamma)
    
    # Results
    results = np.column_stack([scores, C, gamma])
    results_df = pd.DataFrame(results, columns = ['scores', 'C', 'gamma'])
    
    return best_C, best_gamma, results_df


In [157]:
# Define parameters
n_folds = 5
seed = 42

# Unweighted SVM 
# Standarize data
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_sd = scaler.transform(x_train)   

best_c, best_g, results_df = optimum_cvc(x_train_sd, y_train, n_folds, seed, None)

print 'Best C - Weighted: ' + str(best_c)
print 'Best gamma - Weighted: ' + str(best_g)

Best C - Weighted: 1.0
Best gamma - Weighted: 0.001


In [159]:
# Weighted SVM 
best_c_w, best_g_w, results_df_w = optimum_cvc(x_train_sd, y_train, n_folds, seed, 'balanced')

print 'Best C- Unweighted: ' + str(best_c_w)
print 'Best gamma - Unweighted: ' + str(best_g_w)

Best C- Unweighted: 10.0
Best gamma - Unweighted: 1.0


In [158]:
##  Unweighted model: Predict on test sample

# Overall accuracy in Test Sample
start = time.time()
model = SVC(C=best_c, gamma = best_g, class_weight = None)
model.fit(x_train_sd, y_train)
end = time.time()

time_elapsed = end - start
print "Time elapsed - Unweighted SVM:", time_elapsed, "sec"

# Standarize test sample
x_test_sd = scaler.transform(x_test)   

# Prediction
y_pred = model.predict(x_test_sd)

# Overall accuracy
print '\n' '# Test accuracy:', round(model.score(x_test_sd, y_test), 4)

Time elapsed - Unweighted SVM: 21.1899039745 sec

# Test accuracy: 0.3498


In [160]:
##  Weighted model: Predict on test sample

# Overall accuracy in Test Sample
start_w = time.time()
model = SVC(C=best_c_w, gamma = best_g_w, class_weight = 'balanced')
model.fit(x_train_sd, y_train)
end_w = time.time()

time_elapsed_w = end_w - start_w
print "Time elapsed - Weighted SVM:", time_elapsed_w, "sec"

# Prediction
y_pred_w = model.predict(x_test_sd)

# Overall accuracy
print '\n' '# Test accuracy:', round(model.score(x_test_sd, y_test), 4)

Time elapsed - Weighted SVM: 104.380795002 sec

# Test accuracy: 0.3027


In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
C_range = np.logspace(-3, 2, 6)
gamma_range = np.logspace(-3, 2, 6)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(y_train, n_iter=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(x_train, y_train)

----